In [ ]:
import pandas as pd
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/accomodations_to_venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/time_matrix.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/bus_terminals.csv')




In [3]:

venues_to_bus_matrix.to_csv('venues_to_bus_matrix.csv', index=False)